<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task3/chizu_002_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import importlib

if importlib.util.find_spec('datasets') is None:
    !pip install -q torch==2.2.1 transformers==4.39.1 diffusers==0.27.2 torchvision==0.17.1 datasets==2.18.0 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 116.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [5]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task3"
    name = "chizu_002"
    base_model_name="lambdalabs/miniSD-diffusers"
    dataset_path = "/content/drive/MyDrive/dataset/COCO_10000_extracted"
    size=600
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    learning_rate = 2e-05
    resolution = 256
    max_train_steps = 2000
    train_batch_size = 8

    eval_steps = 500

    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=6 # CosineAnnealingLR
    #T_0=6 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    train=True

    prompts = [
        "A curious zebra standing tall in a lush African savanna at sunrise, with acacia trees in the background.",
        "Next to a medieval castle, a regal zebra observes the knights and a drawbridge.",
        "Wearing a scarf, a fashionable giraffe strolls through a bustling city street with skyscrapers.",
        "Running along a sandy beach, a playful giraffe enjoys the palm trees, ocean waves, and a bright sunset.",
        "By a serene lakeside, a relaxed bear drinks water with mountains and a clear blue sky in the background.",
        "In a snowy forest, a cozy bear stands under snow-covered trees, enjoying the gentle snowfall.",
        "Partially hidden in a dense tropical rainforest, an adventurous sheep peeks through leafy plants.",
        "A sleek sheep with modern accessories navigates a futuristic city with flying cars and neon lights.",
    ]

In [6]:
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}
cfg

{'num_workers': 4,
 'project': 'IOAI_Task3',
 'name': 'chizu_002',
 'base_model_name': 'lambdalabs/miniSD-diffusers',
 'dataset_path': '/content/drive/MyDrive/dataset/COCO_10000_extracted',
 'size': 600,
 'scheduler': 'CosineAnnealingLR',
 'learning_rate': 2e-05,
 'resolution': 256,
 'max_train_steps': 2000,
 'train_batch_size': 8,
 'eval_steps': 500,
 'T_max': 6,
 'lr': 0.0001,
 'min_lr': 1e-06,
 'weight_decay': 1e-06,
 'gradient_accumulation_steps': 1,
 'max_grad_norm': 1000,
 'seed': 42,
 'train': True,
 'prompts': ['A curious zebra standing tall in a lush African savanna at sunrise, with acacia trees in the background.',
  'Next to a medieval castle, a regal zebra observes the knights and a drawbridge.',
  'Wearing a scarf, a fashionable giraffe strolls through a bustling city street with skyscrapers.',
  'Running along a sandy beach, a playful giraffe enjoys the palm trees, ocean waves, and a bright sunset.',
  'By a serene lakeside, a relaxed bear drinks water with mountains and 

## Starting the diffusion

Let's start with generating some images with our model to see current state.
Download the model and use diffusers library to make some brand new images.

# Baseline

Below is base retraining for a solution. It would provide some starting point to start the work.

In [7]:
from torch.utils.data import DataLoader
import math
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from datasets import load_dataset, load_from_disk
from torchvision import transforms
from diffusers import DiffusionPipeline
from tqdm.auto import tqdm
import wandb

In [8]:
from google.colab import userdata
wandb.login(key=userdata.get('wandb_token'))

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def terminate_session():
    # Terminate this session

    from google.colab import runtime
    runtime.unassign()

seed_everything(CFG.seed)

In [10]:
def generation_loop(pipe, prompts):
    imgs = []
    for prompt in prompts:
        image = pipe(prompt, width=256, height=256).images[0]
        imgs.append(image)
    return imgs

In [11]:
def main():
    wandb.init(
        name=CFG.name,
        project=CFG.project,
        config=cfg
    )


    ds = load_from_disk(CFG.dataset_path)

    # Extract the individual components
    pipe = DiffusionPipeline.from_pretrained(CFG.base_model_name)
    pipe.to('cuda')
    vae = pipe.vae
    text_encoder = pipe.text_encoder
    tokenizer = pipe.tokenizer
    unet = pipe.unet
    noise_scheduler = pipe.scheduler

    # Freeze vae and text_encoder and set unet to trainable
    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)
    unet.train()

    optimizer = torch.optim.AdamW(unet.parameters(), lr=CFG.learning_rate)

    # Preprocessing the datasets.
    train_transforms = transforms.Compose(
        [
            transforms.Resize(CFG.resolution, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(CFG.resolution),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ]
    )

    # convert dataset to a loader that could be feed during training
    def tokenize_captions(examples, is_train=True):
        captions = examples['text']
        inputs = tokenizer(
            captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        return inputs.input_ids


    def preprocess_train(examples):
        images = [image.convert("RGB") for image in examples['image']]
        examples["pixel_values"] = [train_transforms(image) for image in images]
        examples["input_ids"] = tokenize_captions(examples)
        return examples


    train_dataset = ds.with_transform(preprocess_train)

    def collate_fn(examples):
        pixel_values = torch.stack([example["pixel_values"] for example in examples])
        pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
        input_ids = torch.stack([example["input_ids"] for example in examples])
        return {"pixel_values": pixel_values, "input_ids": input_ids}

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=collate_fn,
        batch_size=CFG.train_batch_size,
        num_workers=0,
    )

    # Training itself
    device = 'cuda'
    weight_dtype = torch.bfloat16

    # Move text_encode and vae to gpu and cast to weight_dtype
    text_encoder.to(device, dtype=weight_dtype)
    vae.to(device, dtype=weight_dtype)
    unet.to(device, dtype=weight_dtype)

    num_train_epochs = math.ceil(CFG.max_train_steps * CFG.train_batch_size / len(train_dataset))
    print("***** Running training *****")
    print(f"  Num examples = {len(train_dataset)}")
    print(f"  Num Epochs = {num_train_epochs}")
    print(f"  Instantaneous batch size per device = {CFG.train_batch_size}")
    print(f"  Total optimization steps = {CFG.max_train_steps}")

    global_step = 0
    initial_global_step = 0

    progress_bar = tqdm(
        range(0, CFG.max_train_steps),
        initial=initial_global_step,
        desc="Steps",
    )

    losses = []

    imgs = generation_loop(pipe, CFG.prompts)
    predictions = [wandb.Image(image, caption=prompt) for image, prompt in zip(imgs, CFG.prompts)]
    wandb.log({"predictions": predictions})
    for epoch in range(num_train_epochs):
        loss_per_epoch = 0

        for step, batch in enumerate(train_dataloader):
            # Convert images to latent space
            latents = vae.encode(batch["pixel_values"].to(weight_dtype).to(device)).latent_dist.sample()
            latents = latents * vae.config.scaling_factor

            # Sample noise that we'll add to the latents
            noise = torch.randn_like(latents)
            batch_size = latents.shape[0]
            # Sample a random timestep for each image
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (batch_size,), device=latents.device)
            timesteps = timesteps.long()

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Get the text embedding for conditioning
            encoder_hidden_states = text_encoder(batch["input_ids"].to('cuda'), return_dict=False)[0]

            # Predict the noise residual and compute loss
            model_pred = unet(noisy_latents, timesteps, encoder_hidden_states, return_dict=False)[0]
            loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")

            # Backpropagate
            loss.backward()
            torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            ###############################################################

            losses.append(loss.item())
            progress_bar.update(1)

            # wandb
            loss_per_epoch += loss.item()

            wandb.log(
                {
                    "Train/epoch": epoch,
                    "Train/step": step,
                    "Train/loss": loss.item(),
                    "Train/global_step": global_step,
                }
            )

            global_step += 1
            progress_bar.set_postfix(average_loss=np.mean(losses[-20:]), step=global_step)
            if global_step >= CFG.max_train_steps:
                break

            # evaluation
            if global_step % CFG.eval_steps == 0:

                imgs = generation_loop(pipe, CFG.prompts)
                predictions = [wandb.Image(image, caption=prompt) for image, prompt in zip(imgs, CFG.prompts)]
                wandb.log({"predictions": predictions})

        loss_per_epoch /= len(train_dataloader)
        wandb.log({"Train/epoch_loss": loss_per_epoch})

    imgs = generation_loop(pipe, CFG.prompts)
    predictions = [wandb.Image(image, caption=prompt) for image, prompt in zip(imgs, CFG.prompts)]
    wandb.log({"predictions": predictions})

In [ ]:
main()

wandb: Currently logged in as: asiatic-cheetah (asiatic-cheetah-a). Use `wandb login --relogin` to force relogin


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

safety_checker/model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
terminate_session()